# Python Concurrency: The Grounded Guide
This notebook demonstrates the five core concepts we discussed:
1. **Concurrency** (Juggling)
2. **Parallelism** (Teamwork)
3. **Threading** (Multiple Actors)
4. **Multiprocessing** (Multiple Brains)
5. **Asyncio** (Pausing & Waiting)

In [ ]:
import time
import asyncio
import multiprocessing
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

def heavy_math(n):
    """A CPU-bound task (Parallelism candidate)"""
    return sum(i * i for i in range(n))

def slow_hardware_call():
    """An I/O-bound task (Concurrency candidate)"""
    time.sleep(2)
    return "Data Ready"

# 1. THE BLOCKING PROBLEM
print("--- 1. Sequential (Blocking) ---")
start = time.time()
heavy_math(10**7)
heavy_math(10**7)
print(f"Total time for two math tasks: {time.time() - start:.2f}s")

--- 1. Sequential (Blocking) ---
Total time for two math tasks: 0.53s


## 1. Threading (Concurrency for I/O)

In [2]:
print("\n--- 2. Threading (Concurrency for I/O) ---")
start = time.time()
with ThreadPoolExecutor(max_workers=2) as executor:
    # We 'delegate' the wait to two threads
    futures = [executor.submit(slow_hardware_call) for _ in range(2)]
    results = [f.result() for f in futures]
print(f"Total time for two hardware calls: {time.time() - start:.2f}s")
print("Notice: It took 2s total, not 4s, because threads 'waited' together.")


--- 2. Threading (Concurrency for I/O) ---
Total time for two hardware calls: 2.00s
Notice: It took 2s total, not 4s, because threads 'waited' together.


## 2. Multiprocessing (Parallelism for Math)

In [9]:
# print("\n--- 3. Multiprocessing (Parallelism for Math) ---")
# start = time.time()
# with ProcessPoolExecutor(max_workers=2) as executor:
#     # We use two DIFFERENT CPU cores
#     futures = [executor.submit(heavy_math, 10**7) for _ in range(2)]
#     results = [f.result() for f in futures]
# print(f"Total time for two math tasks: {time.time() - start:.2f}s")
# print("Notice: This is faster than Cell 2 because it used two CPU cores at once.")

### error: BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.
Ah, the infamous "Pickling Error" of Python Multiprocessing. This is the final boss of Python parallelism, and it’s a classic Mac/Unix hurdle.

The error AttributeError: Can't get attribute 'heavy_math' is happening because you are on a Mac (using the spawn start method). When you start a new process, Python tries to "copy-paste" your function heavy_math into the new process. Because the function is defined inside a Jupyter Notebook cell (the __main__ module), the new process can't "find" it—it's like trying to find a sticky note you left in a book that hasn't been saved yet.

### The Fix: How to make it work in a Notebook
To make multiprocessing work inside Jupyter, the function must be "importable." There are two ways to solve this:

Option 1: The "External File" Trick (Recommended)
This is how you'll do it for your Beacon-dkl-2 project. You put your math functions in a separate .py file and import them.

Create a file named math_utils.py in the same folder.

Put the heavy_math function in there.

Import it in your notebook.


### Option 2: The multiprocess Library (The Shortcut)

There is a more powerful version of the standard library called multiprocess (with an s). It uses a better "pickler" called dill that can actually copy functions directly from your notebook cells without needing an external file.

In [8]:
print("\n--- 3. Multiprocessing (Parallelism for Math) ---")
start = time.time()

# !pip install multiprocess
from multiprocess import Pool # Note the 's' in multiprocess

def heavy_math(n):
    return sum(i * i for i in range(n))

if __name__ == '__main__':
    with Pool(2) as p:
        print(p.map(heavy_math, [10**7, 10**7]))

print(f"Total time for two math tasks: {time.time() - start:.2f}s")
print("Notice: This is faster than Cell 2 because it used two CPU cores at once.")


--- 3. Multiprocessing (Parallelism for Math) ---
[333333283333335000000, 333333283333335000000]
Total time for two math tasks: 0.30s
Notice: This is faster than Cell 2 because it used two CPU cores at once.


## 3. Asyncio -

In [4]:
print("\n--- 4. Asyncio (The Pager System) ---")
async def async_hardware_call():
    await asyncio.sleep(2) # The 'await' bookmark
    return "Data Ready"

async def run_async_test():
    start = time.time()
    # One thread handling two tasks efficiently
    await asyncio.gather(async_hardware_call(), async_hardware_call())
    print(f"Total time for two async calls: {time.time() - start:.2f}s")

# In a notebook, use await directly:
await run_async_test()


--- 4. Asyncio (The Pager System) ---
Total time for two async calls: 2.00s


### One thing we didn't touch on is Context Switching.

Every time the Operating System moves from Thread A to Thread B, it has to:

Stop the CPU.

Save every single variable and register from Thread A to memory.

Load every variable for Thread B from memory.

Start the CPU again.

This "administrative work" isn't free. If you have 1,000 threads, the CPU might spend 40% of its time just doing the paperwork instead of running your code.

This is why Asyncio is so popular for high-speed research: Because it stays on one thread, there is almost zero context switching cost. The "Manager" just moves his eyes to the next line of code.